## 1. Initialization

In [4]:
import sys
import os
import math 

os.chdir("../")
path1 = os.getcwd()

os.chdir("coppeliasim_utils")
path2 = os.getcwd()


sys.path.append(path1)
sys.path.append(path2)

from indy_utils import indydcp_client as client
from coppeliasim_utils import sim

from time import sleep
import time
import json
import threading
import numpy as np
import math
import keyboard
import threading

robot_ip = "192.168.3.19" 
#robot_ip = "192.168.120.163"  # Robot (Indy) IP
robot_name = "NRMK-Indy7"  # Robot name (Indy7)

# Create class object (real robot)
indy_actual = client.IndyDCPClient(robot_ip, robot_name)

# Crate class object (simulation robot)
indy_virtual = sim.simxStart('127.0.0.1', 19997, True, True, 5000, 5)

# Actual robot (Created: True, Not created: False)
# Virtual robot (Created: True, Not created: False)
print("real robot: {0}\nsimulation robot: {1}".format(bool(indy_actual), bool(not indy_virtual)))

real robot: True
simulation robot: True


## 2. Communication reset

In [5]:
# Reset TCP-IP communication (real robot)
# For real robot
indy_actual.connect()
indy_actual.disconnect()


# For simulation robot
sim.simxFinish(indy_virtual)

Connect: Server IP (192.168.0.2)


## 3. Position data acquisition

In [6]:
# For real robot
indy_actual.connect()
j_pos_act = indy_actual.get_joint_pos()
t_pos_act = indy_actual.get_task_pos()

# For simulation robot
handles = []
handles2 = []
j_pos_vir = []
t_pos_vir = []
d_pos_vir = []


for i in range(6):
    object_name = 'joint' + str(i)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        result, jnt_angle = sim.simxGetJointPosition(indy_virtual, handle, sim.simx_opmode_oneshot)
        j_pos_vir.append(jnt_angle/math.pi*180)
        
object_name = 'tool_coordinate'
result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)

result, obj_pos = sim.simxGetObjectPosition(indy_virtual, handle, -1, sim.simx_opmode_oneshot)
result, obj_rot = sim.simxGetObjectOrientation(indy_virtual, handle, -1, sim.simx_opmode_oneshot)

object_name = 'doorhand'
result2, handle2=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)

result2, obj_pos2 = sim.simxGetObjectPosition(indy_virtual, handle2, -1, sim.simx_opmode_oneshot)
result2, obj_rot2 = sim.simxGetObjectOrientation(indy_virtual, handle2, -1, sim.simx_opmode_oneshot)

#그리퍼
handles_grip = []
#hFeederJoint ={-1,-1,-1}
       
for i in range(2):
    object_name = 'gripper_joint' + str(i)
    result3, handle3=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result3 != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        result3, jnt_angle3 = sim.simxGetJointPosition(indy_virtual, handle3, sim.simx_opmode_oneshot)
        j_pos_vir3.append(jnt_angle3/math.pi*180)        
    
########################    
print()

for i in range(3):
    t_pos_vir.append(obj_pos[i])

for i in range(3):
    t_pos_vir.append(obj_rot[i]/np.pi*180)

print("Joint Position (real):", np.round(j_pos_act, 2))
print("Joint Position (simulation):", np.round(j_pos_vir, 2))

print()
print("Task Position (real):", np.round(t_pos_act,2))
print("Task Position (simulation):", np.round(t_pos_vir,2))

t_pos_vir.clear()

for i in range(3):
    t_pos_vir.append(obj_pos2[i])

for i in range(3):
    t_pos_vir.append(obj_rot2[i]/np.pi*180)

print()
print("Door_handle Position (sim):", np.round(t_pos_vir,2))

print("Gripper Joint Position (simulation):", np.round(j_pos_vir3, 2))

Connect: Server IP (192.168.3.19)


Exception: could not get object handle for first joint

## 4. Set position

In [6]:
### For actual robot
indy_actual.connect()

# Zero position
#j_pos_act = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_act = [0, -15, -90, 0, -75, 0]
indy_actual.joint_move_to(j_pos_act)  # Move 6th joint

indy_actual.disconnect()

### For virtual robot
handles = []
#Zero position
#j_pos_vir = [0, 0, 0, 0, 0, 0]

# Home position
j_pos_vir = [0, -15, -90, 0, -75, 0]

for i in range(6):
    object_name = 'joint' + str(i)
    #print(object_name)
    result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
    if result != sim.simx_return_ok:
        raise Exception('could not get object handle for first joint')                
    else:
        handles.append(handle)
        sim.simxSetJointTargetPosition(indy_virtual, handle, j_pos_vir[i]*math.pi/180, sim.simx_opmode_oneshot)

Connect: Server IP (192.168.3.19)


## 5. Synchronization

### 5-1. Synchronization (w non threaded)

### 5-2. Synchronization (w threaded)

In [7]:
def cps_program():
    indy_actual.connect()

    handles = []

    for i in range(6):
            object_name = 'joint' + str(i)    
            #print(object_name)
            result, handle=sim.simxGetObjectHandle(indy_virtual, object_name, sim.simx_opmode_blocking)
            handles.append(handle)

    print(handles)

    isCtrOn = True

    while isCtrOn:
        j_pos = indy_actual.get_joint_pos()
        time.sleep(0.002)

        for i in range(6):
            sim.simxSetJointTargetPosition(indy_virtual, handles[i] , float(j_pos[i])*math.pi/180 ,sim.simx_opmode_oneshot)

        if keyboard.is_pressed("q"):
            isCtrOn = False
            print("Escape the program")

    indy_actual.disconnect()

t = threading.Thread(target=cps_program)
t.start()
print("Main Thread")

Main Thread
Connect: Server IP (192.168.3.19)
[19, 21, 23, 25, 27, 29]


### 5-3. Motion test

In [12]:
j_pos1 = indy_actual.get_joint_pos()
t_pos1 = indy_actual.get_task_pos()
print("j_pos1", j_pos1)
print("t_pos1", t_pos1)

#t_pos1[2] += 0.1

#m미터

a = math.pi/180
print("a : ", a)
pos1 = [ -0, -15, -90, 0, 4.12, 0]
pos2 = [ -0, -34.54, -73.35, 0, 4.12, 0]
pos3 = [t_pos_vir[0], t_pos_vir[1], t_pos_vir[2], t_pos_vir[3], t_pos_vir[4], t_pos_vir[5]]
pos3_j = [89.95, 0.05, -44.13, 9.43, -67.14,-7] #문을 향해 가는 거 조인트
#indy_actual.task_move_to(t_pos1)  # Move along z-axis
#indy_actual.joint_move_to(pos1) 
#sleep(4)
#indy_actual.joint_move_to(pos3) 


#indy_actual.joint_move_to(pos3_j) 
#indy_actual.get_inv_kin(pos3,pos1)

#sleep(4)

def feeder_gripper_on():
    factor = 20
    p = 0.0
    for i in range(factor):
        p=p-0.001
        sim.setJointPosition(hFeederJoint[1],p)
        sim.setJointPosition(hFeederJoint[2],p*2)
        sim.switchThread()
    


feeder_gripper_on()

j_pos1 [89.94999999999796, 0.049999999999999704, -44.130000000001026, 9.429999999999744, -67.13999999999795, -6.999999999999874]
t_pos1 [0.15001957657933526, 0.4749988114681156, 0.900021320126784, 0.005155627709476102, 110.68852805144567, 99.24254271009784]
a :  0.017453292519943295


AttributeError: module 'coppeliasim_utils.sim' has no attribute 'setJointPosition'

Escape the program


In [9]:
indy_actual.go_home()

In [ ]:
indy_actual.stop_motion()

In [21]:
indy_actual.go_zero()

In [ ]:
#gripper
endtool_type = 0
indy_actual.set_endtool_do(endtool_type, 1)
sleep(1)
indy_actual.get_endtool_do(endtool_type)

